<a href="https://colab.research.google.com/github/amanotk/numerical-geophysics/blob/main/notebook/LinearWaveEquation.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

# 線形波動方程式

以下の線形波動方程式の数値解を差分法によって求める．
$$
\frac{\partial^2 f}{\partial t^2} = c^2 \frac{\partial^2 f}{\partial x^2}
$$
ここでは2階の微分方程式を
$$
\begin{aligned}
& \frac{\partial f}{\partial t} + c \frac{\partial g}{\partial x} = 0
\\
& \frac{\partial g}{\partial t} + c \frac{\partial f}{\partial x} = 0
\end{aligned}
$$
のように$(f, g)$の2変数の1階連立微分方程式に変形し，さらに
$$
\begin{aligned}
\frac{\partial}{\partial t} (f + g) + c
\frac{\partial}{\partial x} (f + g) = 0
\\
\frac{\partial}{\partial t} (f - g) - c
\frac{\partial}{\partial x} (f - g) = 0
\end{aligned}
$$
のように対角化した方程式を数値的に解く．  
（簡単に確かめられるように，両式から$g$を消去すれば$f$に関する波動方程式が得られる．）

以下では$c = 1$，$-1 \leq x \leq 1$とし，周期的境界条件を採用する．

In [ ]:
# 準備
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
# フォントサイズ
plt.rcParams["font.size"] = 14

# 初期条件
def set_initial(Nx, sigma, type=0):
  "初期条件を設定する"
  dx = 2/Nx
  xx = (np.arange(Nx+2) - 0.5)*dx - 1
  ff = np.exp(-0.5*xx**2/sigma**2)
  if type == 0:
    gg = np.zeros_like(ff)
  if type == 1:
    gg = ff.copy()
  return xx, ff, gg

def push_lw(f, g, nu, step):
  "Lax-Wendroffスキームによってstep数だけ時間更新する"
  Nx = np.size(f) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  # 特性変数に変換
  u  = f + g
  v  = f - g
  for n in range(step):
    # 更新
    u[ix] = u[ix] - \
      0.5*nu*(u[ix+1] - u[ix-1]) + \
      0.5*nu**2*(u[ix+1] - 2*u[ix] + u[ix-1])
    v[ix] = v[ix] + \
      0.5*nu*(v[ix+1] - v[ix-1]) + \
      0.5*nu**2*(v[ix+1] - 2*v[ix] + v[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]
    # 境界条件
    v[   0] = v[Nx]
    v[Nx+1] = v[ 1]
  # 逆変換
  f[:] = 0.5*(u + v)
  g[:] = 0.5*(u - v)

まずは初期条件として
$$
f(x) = \exp \left( - \frac{x^2}{2 \sigma^2} \right), \quad
g(x) = 0
$$
を採用しよう．このときは左右に同じ振幅で波動が伝播する解が得られる．

In [ ]:
# パラメータ
Nx = 100
Nt = 50
nu = 0.5
dx = 2/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)
w  = np.zeros((Nt, Nx))
t  = np.zeros((Nt,))

# 初期条件
x, u, v = set_initial(Nx, 0.1, 0)

# 数値解
step = 5
for n in range(Nt):
  push_lw(u, v, nu, step)
  w[n,:] = u[+1:-1]
  t[n] = n*step*dt

# 以降プロット
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

## スナップショット
plt.sca(axs[0])
for nn in range(0, Nt//2, 5):
  plt.plot(x[+1:-1], w[nn,:], label='t = {:5.3f}'.format(t[nn]))
plt.xlim(-1.0, 1.0)
plt.xlabel('x')
plt.ylabel('f')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))

## x-tの2次元プロット
plt.sca(axs[1])
t = np.arange(Nt+1) * step*dt
x = 0.5*(x[0:-1] + x[+1:])
T, X = np.broadcast_arrays(t[:,None], x[None,:])
plt.pcolormesh(X, T, w)
plt.xlim(-1.0, 1.0)
plt.xlabel('x')
plt.ylabel('t')

次に初期条件として
$$
f(x) = g(x) = \exp \left( - \frac{x^2}{2 \sigma^2} \right), \quad
$$
としてみよう．このときには右側のみに波動が伝播する．

In [ ]:
# パラメータ
Nx = 100
Nt = 50
nu = 0.5
dx = 2/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)
w  = np.zeros((Nt, Nx))
t  = np.zeros((Nt,))

# 初期条件
x, u, v = set_initial(Nx, 0.1, 1)

# 数値解
step = 5
for n in range(Nt):
  push_lw(u, v, nu, step)
  w[n,:] = u[+1:-1]
  t[n] = n*step*dt

# 以降プロット
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

## スナップショット
plt.sca(axs[0])
for nn in range(0, Nt//2, 5):
  plt.plot(x[+1:-1], w[nn,:], label='t = {:5.3f}'.format(t[nn]))
plt.xlim(-1.0, 1.0)
plt.xlabel('x')
plt.ylabel('f')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))

## x-tの2次元プロット
plt.sca(axs[1])
t = np.arange(Nt+1) * step*dt
x = 0.5*(x[0:-1] + x[+1:])
T, X = np.broadcast_arrays(t[:,None], x[None,:])
plt.pcolormesh(X, T, w)
plt.xlim(-1.0, 1.0)
plt.xlabel('x')
plt.ylabel('t')